In [1]:
# 让notebook宽屏显示
from IPython.display import display, HTML
display(HTML('<style>.container{width:100% !important;}</style>'))

In [2]:
import pandas as pd
import numpy as np
import datetime
import xgboost as xgb
from xgboost import plot_importance
import operator
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

/Users/limeng/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_auth_info = pd.read_csv('./data/AI_Risk_Train_V3.0/train_auth_info.csv', low_memory=False)           
train_bankcard_info = pd.read_csv('./data/AI_Risk_Train_V3.0/train_bankcard_info.csv', low_memory=False)
train_credit_info = pd.read_csv('./data/AI_Risk_Train_V3.0/train_credit_info.csv', low_memory=False)
train_order_info = pd.read_csv('./data/AI_Risk_Train_V3.0/train_order_info.csv', low_memory=False)
train_recieve_addr_info = pd.read_csv('./data/AI_Risk_Train_V3.0/train_recieve_addr_info.csv', low_memory=False)
train_user_info = pd.read_csv('./data/AI_Risk_Train_V3.0/train_user_info.csv', low_memory=False)
train_target = pd.read_csv('./data/AI_Risk_Train_V3.0/train_target.csv', low_memory=False)

In [4]:
def setlen(group):
    return len(set(group))

def return_set(group):
    return set(group)

In [5]:
'''
身份证账号是否为空
认证时间是否为空
电话号码是否为空
是否所有信息都为空，除了 id
是否所有信息都不为空
认证时间和身份证是否同时为空
认证时间和手机号码是否同时为空
'''

def auth_info(data):
    #身份证账号是否为空,空值为0
    data['id_card_isnull'] = [1 if type(i) == str else 0 for i in data.id_card]
    #电话号码是否为空
    data['phone_isnull'] = [1 if type(i) == str else 0 for i in data.phone]
    #认证时间是否为空
    data['auth_time_isnull'] = [1 if type(i) == str else 0 for i in data.auth_time]
    #train_auth_info非空数量
    data['auth_info_notnull_num']=data['id_card_isnull']+data['phone_isnull']+data['auth_time_isnull']
    
    return data[['id_card_isnull', 'phone_isnull', 'auth_time_isnull', 'auth_info_notnull_num','id']]

In [6]:
'''
用户有多少条记录
用户有多少个不同的手机号码
用户的储蓄卡的数量
用户的信用卡的数量
用户是否有信用卡
用户有几种不同类型的银行卡
用户银行卡数量是否大于 6
用户是否只有一张银行卡
'''
def bankcard_info(data):
    #card_1储蓄卡，card_2信用卡
    data['card_1'] = [1 if i == '储蓄卡' else 0 for i in data['card_type']]
    data['card_2'] = [1 if i == '信用卡' else 0 for i in data['card_type']]
    #用户储蓄卡/信用卡的数量
    card_1_cnt = pd.pivot_table(data, index='id', values='card_1', aggfunc='sum').reset_index().rename(columns={'card_1': 'card_1_cnt'})
    data = data.merge(card_1_cnt, on='id', how='left')
    card_2_cnt = pd.pivot_table(data, index='id', values='card_2', aggfunc='sum').reset_index().rename(columns={'card_2': 'card_2_cnt'})
    data = data.merge(card_2_cnt, on='id', how='left')
    #各家银行用户数量
    bank_cnt = pd.pivot_table(data, index='bank_name', values='tail_num', aggfunc='count').reset_index().rename(columns={'tail_num': 'bank_cnt'})
    #用户开卡银行数量/银行卡数量/手机号数量/使用银行数量
    id_bank_cnt = pd.pivot_table(data, index='id', values='bank_name', aggfunc='count').reset_index().rename(columns={'bank_name': 'id_bank_cnt'})
    id_phone_set = pd.pivot_table(data, index='id', values='phone', aggfunc=setlen).reset_index().rename(columns={'phone': 'id_phone_set'})
    id_card_set = pd.pivot_table(data, index='id', values='card_type', aggfunc=setlen).reset_index().rename(columns={'card_type': 'id_card_set'})
    id_bank_set = pd.pivot_table(data, index='id', values='bank_name', aggfunc=setlen).reset_index().rename(columns={'bank_name': 'id_bank_set'})
    #id_phone_set=data.groupby('id').agg({'phone':pd.Series.nunique}).rename(columns={'phone': 'id_phone_set'})
    #id_card_set=data.groupby('id').agg({'card_type':pd.Series.nunique}).rename(columns={'card_type': 'id_card_set'})
    #id_bank_set=data.groupby('id').agg({'bank_name':pd.Series.nunique}).rename(columns={'bank_name': 'id_bank_set'})
    
    
    
    
    data = data.merge(bank_cnt, on='bank_name', how='left')
    data = data.merge(id_bank_cnt, on='id', how='left')
    data = data.merge(id_phone_set, on='id', how='left')
    data = data.merge(id_card_set, on='id', how='left')  # ?
    data = data.merge(id_bank_set, on='id', how='left')  # ?
    return data[['id', 'card_1_cnt', 'card_2_cnt', 'id_bank_cnt', 'id_phone_set', 'id_card_set', 'id_bank_set']].drop_duplicates(['id'])

In [7]:
'''
用户信用积分
用户额度是否为 0
用户已使用的额度
用户的信用额度
用户剩余的额度
用户额度使用率
用户额度排名
是否所有信息都为空，除了 id
是否所有信息都为 0，除了 id
信用额度是否为 0
信用积分是否为 0
用户是否还有剩余的额度
'''
def credit_info(data):
    #剩余额度
    data['q_o'] = data['quota'] - data['overdraft']
    #处理空值
    data['quota'] = [1 if i is np.nan else i for i in data['quota']]
    data['overdraft'] = [1 if i is np.nan else i for i in data['overdraft']]
    #用户额度使用率
    data['q/o'] = data[['quota', 'overdraft']].apply(lambda x: 0 if x.quota == 0 else x.overdraft/x.quota, axis=1)
    return data.drop_duplicates(['id'])

In [8]:
'''
用户记录是否有除 id 外都为空
用户关于商品单价的统计特征
用户关于订单金额的统计特征
支付方式的离散特征
订单状态的离散特征
'''
def order_info(data):
    id_sample = data.drop_duplicates(['id'])[['id']]

    data = data.drop_duplicates()
    order_info_amt = data[['amt_order']]
    order_info_amt = order_info_amt[order_info_amt['amt_order'].notnull()]
    order_info_amt = order_info_amt[order_info_amt['amt_order'] != 'null']
    order_info_amt['amt_order'] = [float(index) for index in order_info_amt['amt_order']]
    mean = order_info_amt['amt_order'].mean()
    data['amt_order'] = data['amt_order'].fillna(mean)
    data['amt_order'] = [mean if index == 'null' else index for index in data['amt_order']]
    data['amt_order'] = [float(index) for index in data['amt_order']]

    data['pay_way_1'] = [1 if i == '在线支付' else 0 for i in data['type_pay']]
    way1_cnt = pd.pivot_table(data, index='id', values='pay_way_1', aggfunc='sum').reset_index().rename(columns={'pay_way_1': 'way1_cnt'})
    id_sample = id_sample.merge(way1_cnt, on='id', how='left')
    data['pay_way_2'] = [1 if i == '货到付款' else 0 for i in data['type_pay']]
    way2_cnt = pd.pivot_table(data, index='id', values='pay_way_2', aggfunc='sum').reset_index().rename(columns={'pay_way_2': 'way2_cnt'})
    id_sample = id_sample.merge(way2_cnt, on='id', how='left')

    '''统计计数特征'''
    f9 = pd.pivot_table(data[['id', 'type_pay']], index='id', values='type_pay', aggfunc=setlen).reset_index().rename(columns={'type_pay': 'id_type_pay_set'})
    id_sample = id_sample.merge(f9, on='id', how='left')
    f10 = pd.pivot_table(data[['id', 'sts_order']], index='id', values='sts_order', aggfunc=setlen).reset_index().rename(columns={'sts_order': 'id_sts_order_set'})
    id_sample = id_sample.merge(f10, on='id', how='left')
    f11 = pd.pivot_table(data[['id', 'phone']], index='id', values='phone', aggfunc=setlen).reset_index().rename(columns={'phone': 'id_phone_set'})
    id_sample = id_sample.merge(f11, on='id', how='left')

    '''其他特征'''
    data['sts_order'] = data['sts_order'].fillna('0')
    data['wan_cheng'] = [1 if ('完成' in i) else 0 for i in data['sts_order']]
    wan_cheng_cnt = pd.pivot_table(data, index='id', values='wan_cheng', aggfunc='sum').reset_index().rename(columns={'wan_cheng': 'wan_cheng_cnt'})
    id_sample = id_sample.merge(wan_cheng_cnt, on='id', how='left')
    data['cheng_gong'] = [1 if '成功' in i else 0 for i in data['sts_order']]
    print(data['cheng_gong'])
    cheng_gong_cnt = pd.pivot_table(data, index='id', values='cheng_gong', aggfunc='sum').reset_index().rename(columns={'cheng_gong': 'cheng_gong_cnt'})
    id_sample = id_sample.merge(cheng_gong_cnt, on='id', how='left')
    data['qu_xiao'] = [1 if '取消' in i else 0 for i in data['sts_order']]
    qu_xiao_cnt = pd.pivot_table(data, index='id', values='qu_xiao', aggfunc='sum').reset_index().rename(columns={'qu_xiao': 'qu_xiao_cnt'})
    id_sample = id_sample.merge(qu_xiao_cnt, on='id', how='left')

    return id_sample.drop_duplicates(['id'])

In [9]:
'''
用户记录中是否有除 id 外都为空
'addr_id', 'region', 'phone', 'fix_phone', 'receiver_md5'是否同时为空
用户的记录数
用户收获地址中的省份离散特征
用户收获地址中有多少不同的省份
'''
def recieve_addr_info(data):
    province = {'甘肃', '云南', '贵州', '河南', '黑龙', '香港', '北京', '湖南', '江苏', '青海', '宁夏', '内蒙', '浙江', '吉林', '海南', '福建', '重庆', '台湾', '陕西', '湖北', '江西', '辽宁', '山西', '西藏', '广东', '安徽', '四川', '河北', '山东', '上海',
                '广西', '新疆', '天津', 'null'}
    data['province'] = data[['region']].apply(lambda x: 'null' if x.region is np.nan else x.region[0:2], axis=1)
    city_set = pd.pivot_table(data, index='id', values='province', aggfunc=return_set).reset_index()
    for string in list(province):
        city_set[string] = [1 if string in index else 0 for index in city_set['province']]
    city_set['province_p'] = city_set[['province']].apply(lambda x: x.province.clear() if 'null' in x.province else x.province, axis=1)
    city_set['province_len'] = [0 if index is None else len(index) for index in city_set['province']]

    data['phone_isnull'] = [0 if type(i) == float else 1 for i in data.phone]
    data['fix_phone_isnull'] = [1 if type(i) == str else 0 for i in data.fix_phone]
    id_phone_set = pd.pivot_table(data[['id', 'phone']], index='id', values='phone', aggfunc=setlen).reset_index().rename(columns={'phone': 'id_phone_set'})
    data = data.merge(id_phone_set, on='id', how='left')
    data = data.merge(city_set, on='id', how='left')

    return data[['id', 'phone_isnull', 'fix_phone_isnull', 'id_phone_set', 'province_len']].drop_duplicates(['id'])

In [10]:
'''
用户生日是否是“0000-00-00”
用户性别的 one-hot 编码
用户婚姻状况的 one-hot 编码
用户会员等级的 one-hot 编码
用户是否绑定 QQ
用户是否绑定微信号
用户学历是否是“硕士、其它、博士”
用户身份证号是否为空
用户会员收入的 one-hot 编码
'''
def user_info(data):
    id_sample = data[['id']]
    degree = ['本科', '初中', '中专', '其他', '硕士', '大专', '博士', '高中']
    for index in degree:
        id_sample[index] = [1 if index == string else 0 for string in data['degree']]

    id_sample['sex_isnull'] = [0 if type(index) == float else 1 for index in data['sex']]
    id_sample['sex1'] = [1 if index == '保密' else 0 for index in data['sex']]
    id_sample['sex2'] = [1 if index == '男' else 0 for index in data['sex']]
    id_sample['sex3'] = [1 if index == '女' else 0 for index in data['sex']]

    id_sample['0000-00-00'] = [1 if index == '0000-00-00' else 0 for index in data['birthday']]

    id_sample['merriage1'] = [1 if index == '未婚' else 0 for index in data['merriage']]
    id_sample['merriage2'] = [1 if index == '已婚' else 0 for index in data['merriage']]
    id_sample['merriage3'] = [1 if index == '保密' else 0 for index in data['merriage']]

    id_sample['income_isnull'] = [1 if type(index) == str else 0 for index in data['income']]
    id_sample['income1'] = [1 if index == '4000-5999元' else 0 for index in data['income']]
    id_sample['income2'] = [1 if index == '8000元以上' else 0 for index in data['income']]
    id_sample['income3'] = [1 if index == '2000-3999元' else 0 for index in data['income']]
    id_sample['income4'] = [1 if index == '6000-7999元' else 0 for index in data['income']]
    id_sample['income5'] = [1 if index == '2000元以下' else 0 for index in data['income']]

    id_sample['id_card_isnull'] = [1 if type(index) == str else 0 for index in data['id_card']]

    id_sample['qq_bound_one'] = [1 if index == '已绑定' else 0 for index in data['qq_bound']]
    id_sample['qq_bound_two'] = [1 if index == '未绑定' else 0 for index in data['qq_bound']]

    id_sample['wechat_bound_one'] = [1 if index == '已绑定' else 0 for index in data['wechat_bound']]
    id_sample['wechat_bound_two'] = [1 if index == '未绑定' else 0 for index in data['wechat_bound']]

    id_sample['account_grade_one'] = [1 if index == '注册会员' else 0 for index in data['account_grade']]
    id_sample['account_grade_two'] = [1 if index == '铜牌会员' else 0 for index in data['account_grade']]
    id_sample['account_grade_three'] = [1 if index == '银牌会员' else 0 for index in data['account_grade']]
    id_sample['account_grade_four'] = [1 if index == '金牌会员' else 0 for index in data['account_grade']]
    id_sample['account_grade_five'] = [1 if index == '钻石会员' else 0 for index in data['account_grade']]
    return id_sample.drop_duplicates(['id'])

In [11]:
'''
用户年龄
用户注册天数
用户借贷日期是否早于注册日期
下订单时间与注册时间的天数差的最大、最小、平均
'''
def days_feature(auth, order, appl):
    data = auth.merge(appl, on='id', how='left')
    data['auth_time'] = data[['appl_sbm_tm', 'auth_time']].apply(lambda x: x.appl_sbm_tm[:10] if x.auth_time == '0000-00-00' else x.auth_time, axis=1)
    data['auth_time'] = data[['appl_sbm_tm', 'auth_time']].apply(lambda x: x.appl_sbm_tm[:10] if x.auth_time is np.nan else x.auth_time, axis=1)
    #认证时间-贷款申请时间
    data['days'] = data[['auth_time', 'appl_sbm_tm']].apply(lambda x: (datetime.datetime.strptime(x.appl_sbm_tm[:10], '%Y-%m-%d') - datetime.datetime.strptime(x.auth_time[:10], '%Y-%m-%d')).days, axis=1)
    data['days_is_neg'] = [1 if i > 0 else 0 for i in data['days']]
    data['auth_year'] = data[['auth_time']].apply(lambda x: int(x.auth_time[:4]), axis=1)
    data['appl_year'] = data[['appl_sbm_tm']].apply(lambda x: int(x.appl_sbm_tm[:4]), axis=1)
    data['years'] = data['appl_year'] - data['auth_year']
    data['years_is_neg'] =data[['years']].apply(lambda x: 1 if x.years > 0 else 0, axis=1)
    print("OK")
    return data[['id', 'days', 'days_is_neg', 'years', 'years_is_neg']].drop_duplicates(['id'])

In [12]:
def auth_order(auth, order):
    data = auth.merge(order, on='id', how='left')
    data['auth_time'] = [i if type(i) == str else '0001-01-01' for i in data['auth_time']]
    data['auth_time'] = ['0001-01-01' if i == '0000-00-00' else i for i in data['auth_time']]
    data['auth_time'] = ['0001-01-01' if i == 0 else i for i in data['auth_time']]
    data['time_order'] = [i if type(i) == str else '0001-01-01 00:00:00' for i in data['time_order']]
    data['time_order'] = [i if len(i) > 16 else '0001-01-01 00:00:00' for i in data['time_order']]

    data['time_days'] = data[['auth_time', 'time_order']].apply(
        lambda x: abs((datetime.datetime.strptime(x.time_order, '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(x.auth_time, '%Y-%m-%d')).days), axis=1)
    data['time_days'] = [i if ((i < 50000) & (i > 0)) else -1 for i in data['time_days']]
    time_days_mean = pd.pivot_table(data[['id', 'time_days']], index='id', values='time_days', aggfunc='mean').reset_index().rename(columns={'time_days': 'time_days_mean'})
    auth = auth.merge(time_days_mean, on='id', how='left')
    auth['time_days_mean_is_neg'] = [1 if i > 0 else 0 for i in auth['time_days_mean']]
    return auth[['id', 'time_days_mean', 'time_days_mean_is_neg']]

In [13]:
'''训练集读取、提特征'''
train_auth_info = pd.read_csv('./data/AI_Risk_Train_V3.0/train_auth_info.csv', low_memory=False)
f_train_auth_info = auth_info(train_auth_info)
train_bankcard_info = pd.read_csv('./data/AI_Risk_Train_V3.0/train_bankcard_info.csv', low_memory=False)
f_train_bankcard_info = bankcard_info(train_bankcard_info)
train_credit_info = pd.read_csv('./data/AI_Risk_Train_V3.0/train_credit_info.csv', low_memory=False)
f_train_credit_info = credit_info(train_credit_info)
train_order_info = pd.read_csv('./data/AI_Risk_Train_V3.0/train_order_info.csv', low_memory=False)
f_train_order_info = order_info(train_order_info)
train_recieve_addr_info = pd.read_csv('./data/AI_Risk_Train_V3.0/train_recieve_addr_info.csv', low_memory=False)
f_train_recieve_addr_info = recieve_addr_info(train_recieve_addr_info)
train_user_info = pd.read_csv('./data/AI_Risk_Train_V3.0/train_user_info.csv', low_memory=False)
f_train_user_info = user_info(train_user_info)
train_target = pd.read_csv('./data/AI_Risk_Train_V3.0/train_target.csv', low_memory=False)
feature_l = train_target[['id', 'target']]
day_minus = days_feature(train_auth_info[['id', 'auth_time']], train_order_info[['id', 'time_order']], train_target[['id', 'appl_sbm_tm']])
auth_or = auth_order(train_auth_info[['id', 'auth_time']], train_order_info[['id', 'time_order']])

0          0
1          0
2          0
3          0
4          0
          ..
2415640    0
2415641    0
2415642    0
2415643    0
2415644    0
Name: cheng_gong, Length: 2343902, dtype: int64
OK


In [14]:
'''划分验证集'''
feature_l['date'] = [index.replace('-', '') for index in train_target['appl_sbm_tm']]
feature_l['date'] = [index.split(' ')[0][0:6] for index in feature_l['date']]
validation_train = feature_l[feature_l['date'] != '201704'][['target', 'id','date']]#暂时保留date，用于划分时间进行不同时间点的特征训练
validation_test = feature_l[feature_l['date'] == '201704'][['target', 'id']]

In [15]:
'''validation_train'''
validation_train = validation_train.merge(f_train_auth_info, on='id', how='left')
validation_train = validation_train.merge(f_train_bankcard_info, on='id', how='left')
validation_train = validation_train.merge(f_train_credit_info, on='id', how='left')
validation_train = validation_train.merge(f_train_order_info, on='id', how='left')
validation_train = validation_train.merge(f_train_recieve_addr_info, on='id', how='left')
validation_train = validation_train.merge(f_train_user_info, on='id', how='left')
validation_train = validation_train.merge(day_minus, on='id', how='left')
validation_train = validation_train.merge(auth_or, on='id', how='left')

'''validation_test'''
validation_test = validation_test.merge(f_train_auth_info, on='id', how='left')
validation_test = validation_test.merge(f_train_bankcard_info, on='id', how='left')
validation_test = validation_test.merge(f_train_credit_info, on='id', how='left')
validation_test = validation_test.merge(f_train_order_info, on='id', how='left')
validation_test = validation_test.merge(f_train_recieve_addr_info, on='id', how='left')
validation_test = validation_test.merge(f_train_user_info, on='id', how='left')
validation_test = validation_test.merge(day_minus, on='id', how='left')
validation_test = validation_test.merge(auth_or, on='id', how='left')
print(validation_test.shape)

(32167, 67)


In [16]:
validation_train

,target,id,date,id_card_isnull_x,phone_isnull_x,auth_time_isnull,auth_info_notnull_num,card_1_cnt,card_2_cnt,id_bank_cnt,...,account_grade_two,account_grade_three,account_grade_four,account_grade_five,days,days_is_neg,years,years_is_neg,time_days_mean,time_days_mean_is_neg
0,0,20160608150003808365,201606,1,1,1,3,4,2,6,...,0,0,0,0,129,1,0,0,318.461538,1
1,0,20160427110002485018,201604,1,1,1,3,3,2,5,...,1,0,0,0,-417,0,-1,0,1103.000000,1
2,0,593852820159991816,201703,0,1,0,1,2,3,5,...,0,0,0,0,0,0,0,0,-1.000000,0
3,0,506970882778861832,201611,1,1,1,3,1,2,3,...,0,0,0,0,-380,0,-1,0,-1.000000,0
4,0,584688602999427336,201703,1,1,1,3,1,0,1,...,0,1,0,0,56,1,0,0,-1.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88757,0,576751542485389576,201703,1,1,1,3,1,0,1,...,1,0,0,0,228,1,1,1,48.200000,1
88758,0,590710366225305608,201703,0,1,0,1,2,0,2,...,0,0,0,0,0,0,0,0,-1.000000,0
88759,0,578853574977523976,201703,0,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,-1.000000,0
88760,0,492795832240705800,201611,1,0,0,1,0,1,1,...,1,0,0,0,0,0,0,0,-1.000000,0


In [17]:
validation_test

,target,id,id_card_isnull_x,phone_isnull_x,auth_time_isnull,auth_info_notnull_num,card_1_cnt,card_2_cnt,id_bank_cnt,id_phone_set_x,...,account_grade_two,account_grade_three,account_grade_four,account_grade_five,days,days_is_neg,years,years_is_neg,time_days_mean,time_days_mean_is_neg
0,0,600689284231467016,1,1,1,3,2,0,2,1,...,0,0,0,0,34,1,0,0,410.0,1
1,0,607748277420429320,0,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,-1.0,0
2,0,614829617332621576,1,1,0,2,2,0,2,1,...,0,0,0,0,0,0,0,0,-1.0,0
3,0,596469748414288136,1,1,1,3,2,0,2,1,...,0,0,0,0,108,1,1,1,-1.0,0
4,0,604455889608380680,0,1,0,1,2,0,2,1,...,0,0,0,0,0,0,0,0,-1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32162,0,613672911420526600,1,1,1,3,1,0,1,1,...,1,0,0,0,-56,0,0,0,-1.0,0
32163,0,611324920504258824,0,1,0,1,2,2,4,1,...,0,0,0,0,0,0,0,0,-1.0,0
32164,0,610017602076413960,1,1,1,3,2,0,2,1,...,0,0,0,0,-8,0,0,0,-1.0,0
32165,0,613482618993774856,0,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,-1.0,0


In [46]:
validation_train.to_csv('./validation_train_9.csv')
validation_test.to_csv('./validation_test_9.csv')